# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2


from prefect import Flow, task
from prefect.tasks.shell import ShellTask
from prefect.schedules.clocks import IntervalClock
from prefect.schedules import Schedule
from dask.distributed import Client
from prefect.executors import LocalDaskExecutor

from datetime import datetime, timedelta
import pendulum
import os
import sys
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data import io

cwd = os.getcwd().split("/")[-1]
if cwd == "notebooks":
    os.chdir("../")

In [2]:
executor=LocalDaskExecutor()

# Scrape Events Flow

In [3]:
event_schedule = Schedule([IntervalClock(start_date=pendulum.datetime(2020, 12, 13, 6, tz="America/Los_Angeles"),
                               interval=timedelta(days=7))])

shell_task = ShellTask(helper_script="cd scrapy_ufcstats")

with Flow("Scrape Events", executor=executor, schedule=event_schedule) as f: 
    date_today = datetime.now().date()
    scrapy_crawl_events_cmd = f"scrapy crawl events -O ../data/events/event_urls_{date_today}.csv"
    shell_task(command=scrapy_crawl_events_cmd)
    
    csv_file_names = io.get_filenames(path="data/events/", extension="csv")
    csv_file_names_arr = io.arr_to_series(csv_file_names)
    string_dates_arr = io.get_string_date(csv_file_names_arr)
    io.get_max_date(string_dates_arr)
    
f.register("scrape-ufcstats")


Result check: OK
Flow URL: https://cloud.prefect.io/jasonminsookim-gmail-com-s-account/flow/ca3bd7bb-4682-46d9-b4aa-6dab42a2cccd
 └── ID: 3c693644-37df-428e-998d-7923c3311270
 └── Project: scrape-ufcstats
 └── Labels: ['Mac-mini.local']


'3c693644-37df-428e-998d-7923c3311270'

In [1]:
with Flow("Test", executor=executor) as f:
    conn = io.connect_postgres()
# Registers the Flow to the Prefect project.
#f.register("scrape-ufcstats")
f.run()



NameError: name 'Flow' is not defined